## 0. Load data

In [1]:
!ls ../data

bitcoin_4h_data.csv eth_daily_data.csv


In [24]:
import pandas as pd

df = pd.read_csv("../data/bitcoin_4h_data.csv")
df.head()

,Date,Timestamp,Price
0,2023-11-01 00:00:00,1698796808027,34672.289284
1,2023-11-01 04:00:00,1698811212766,34451.560561
2,2023-11-01 08:00:00,1698825600129,34491.825779
3,2023-11-01 12:00:00,1698840021523,34410.274863
4,2023-11-01 16:00:00,1698854409327,34254.653925


In [25]:
df["Date"] = pd.to_datetime(df["Date"])
df.head()

,Date,Timestamp,Price
0,2023-11-01 00:00:00,1698796808027,34672.289284
1,2023-11-01 04:00:00,1698811212766,34451.560561
2,2023-11-01 08:00:00,1698825600129,34491.825779
3,2023-11-01 12:00:00,1698840021523,34410.274863
4,2023-11-01 16:00:00,1698854409327,34254.653925


In [26]:
df.set_index("Date", inplace = True)
df.head()

,Timestamp,Price
Date,,
2023-11-01 00:00:00,1698796808027,34672.289284
2023-11-01 04:00:00,1698811212766,34451.560561
2023-11-01 08:00:00,1698825600129,34491.825779
2023-11-01 12:00:00,1698840021523,34410.274863
2023-11-01 16:00:00,1698854409327,34254.653925


In [27]:
df.rename(columns = {"Price" : "Close"}, inplace = True)
df.head()

,Timestamp,Close
Date,,
2023-11-01 00:00:00,1698796808027,34672.289284
2023-11-01 04:00:00,1698811212766,34451.560561
2023-11-01 08:00:00,1698825600129,34491.825779
2023-11-01 12:00:00,1698840021523,34410.274863
2023-11-01 16:00:00,1698854409327,34254.653925


In [28]:
data = df[["Close"]]
data.head()

,Close
Date,
2023-11-01 00:00:00,34672.289284
2023-11-01 04:00:00,34451.560561
2023-11-01 08:00:00,34491.825779
2023-11-01 12:00:00,34410.274863
2023-11-01 16:00:00,34254.653925


In [50]:
len(data)

2010

## 1. Backtest Engine

1. **Load data** 
2. **Apply strategies and Execute orders**
3. **Record results**

### Key Methods:

1. `__init__()`:
    - **take in the testing data**
    - **take in a strategy object**
    - **initialize capital balance**
2. `run()`:
    - **apply strategy to generate signals**
    - **execute buy/sell trades** 
3. `get_results()`:
    - **analyze strategy's performance**

### 1.1 Backtest Engine Object (`backtest_engine.py`)

In [ ]:

class BacktestEngine:

    def __init__(self, data, strategy, initial_cash = 1000):
        """
        Initialize the backtest engine
        
        @param data: dataframe has columns of "Date" and "Close", generated by csv_loader
        @param strategy: strategy object (e.g. RSIStrategy object)
        @param initial_cash: default 1000
        """
        
        self.data = data
        self.strategy = strategy
        self.initial_cash = initial_cash
        
        self.cash_stack = [initial_cash * 0.2] * 5
        self.position_stack = []
        self.portfolio_value_list = [] #Track portfolio value over time

    
    def run(self):
        """
        Generate trading signals and Execute trades based on signals
        Protfolio value will be recorded over time
        """

        #Generate trading signals for each row of price
        signal_list = self.strategy.generate_signal(self.data)

        for i in range(len(self.data)):
            signal = signal_list[i]
            price = self.data["Close"].iloc[i]

            if signal == 1 and len(self.cash_stack) > 0:  # buy signal
                cash = self.cash_stack.pop()
                shares_to_buy = cash / price
                
                self.position_stack.append(shares_to_buy)

            elif signal == -1 and len(self.position_stack) > 0:   #sell signal
                shares_to_sell = self.position_stack.pop()
                cash = price * shares_to_sell

                self.cash_stack.append(cash)

            # calculate current portfolio value
            current_portfolio_value = sum(self.cash_stack) + sum(self.position_stack) * price
            self.portfolio_value_list.append(current_portfolio_value)

    def get_results(self):
        """
        retrieve the portfolio values over the backtesting period
        
        @return: A list of portfolio values at each time step
        """
        return self.portfolio_value_list


    def calculate_performance(self):
        """
        calculate the total return

        @return: a dictionary that contains total return information
        """
        final_value = self.portfolio_value_list[-1]

        total_return = (final_value - self.initial_cash) / self.initial_cash

        return {"Total Return": total_return}
        

In [9]:
1123*0.2

224.60000000000002

## 2. Strategy Module

1. `__init__()`:
   - **load the trained machine learning model**
   - **initilize the strategy parameters (such as RSI overbought threshold, moving average periods etc)**
2. `generate_signal()`:
   - **take in the historical data**
   - **return buy/sell signal based on input data**
4. other helper methods:
   - **helper methods such as calculating RSI, MACD etc, internal to the strategy and not directly called by backtest engine**

### 2.1 RSI Strategy Module

In [44]:
import talib
import numpy as np

In [17]:
data.head()

,Close
Date,
2023-11-01 00:00:00,34672.289284
2023-11-01 04:00:00,34451.560561
2023-11-01 08:00:00,34491.825779
2023-11-01 12:00:00,34410.274863
2023-11-01 16:00:00,34254.653925


In [30]:
rsi_series = talib.RSI(data["Close"], timeperiod = 14)   #data["Close"] is a pandas series
type(rsi_series)

pandas.core.series.Series

In [31]:
rsi_array = talib.RSI(data["Close"].values, timeperiod = 14)    # data["Close"].values is a numpy array
type(rsi_array)

numpy.ndarray

In [34]:
rsi_array

array([        nan,         nan,         nan, ..., 36.53462011,
       34.30265935, 33.67864444])

In [47]:
np.nan > 100, np.nan<30

(False, False)

In [49]:
pd.isna(rsi_array[0])

True

In [15]:
import talib
import numpy as np

class RSIStrategy:

    def __init__(self, overbought_threshold = 70, oversold_threshold = 30, timeperiod = 14):
        """
        Initialize the RSI strategy object
        
        @param overbought_threshold: default value 70, sell if RSI above it
        @param oversold_threshold: default value 30, buy if RSI below it
        @param timeperiod: default value 14, the period to calculate RSI
        """

        self.overbought_threshold = overbought_threshold
        self.oversold_threshold = oversold_threshold
        self.timeperiod = timeperiod

    def generate_signal(self, data):
        """
        Generate a list of trading signals based on RSI values
        
        @param data: a data frame with columns of "Date" and "Close", it is generated by csv_loader
        
        @return: a list (signal_list) that contains trading signals: 1 for buy, -1 for sell, and 0 for hold
        """
        signal_list = []

        rsi_array = talib.RSI(data["Close"].values, timeperiod = self.timeperiod)

        
        for rsi in rsi_array:
            
            if rsi < self.oversold_threshold:  # buy signal
                signal_list.append(1)
                
            elif rsi > self.overbought_threshold:  # sell signal
                signal_list.append(-1)

            else:
                singal_list.append(0)   # hold your position
            
        
        return signal_list

## 3. main.py

1. **Load csv data**
2. **Initialize strategy object**
3. **Initialize backtest engine object by taking in data and strategy**
4. **Run backtest engine**
5. **Get backtest results**

In [ ]:


if __name__ == "__main__":

    # load csv data from data/bitcoin_4h_data.csv
    # use the csv_loader.py in data_loader directory to covert the csv data to a df with columns "Date" and "Close"
    from data_loader.csv_loader import load_csv_data
    data = load_csv_data("data/bitcoin_4h_data.csv", source = "coingecko")


    # Initialize RSI strategy object
    from strategies.rsi_strategy import RSIStrategy
    strategy = RSIStrategy()


    # Initialize backtest engine
    from core.backtest_engine import BacktestEngine
    backtest_engine = BacktestEngine(data, strategy, initial_balance = 1000)


    # Run backtest engine
    backtest_engine.run()


    #calculate and print the performance
    performance = backtest_engine.calculate_performance()
    print(performance)



In [3]:
!ls ..

LICENSE     config      data        notebooks   tests
README.md   core        data_loader strategies


In [4]:
!ls ../data_loader

csv_loader.py               data_generator_yf.py
data_generator_coingecko.py


In [2]:
!ls ../strategies

RSIStrategy.py  rsi_strategy.py


In [1]:
!ls ../core

backtest_engine.py
